<a href="https://colab.research.google.com/github/TomBeagley/MEng-FYP/blob/main/FINAL_predictions_%26_error_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries

In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

Import all training and validation data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Create testing data set (images and labels) (all pre-processed in "pre-processing" notebook)

In [ ]:
os.chdir("/content/gdrive/My Drive/FYP Data/FINAL DATA SET/labelsTe")

all_files = os.listdir()
#print(all_files)
#print()

# get all nifti files (images)
nii_files = filter(lambda x: x[-4:] == '.nii', all_files)
nii_files = list(nii_files)
nii_files = sorted(nii_files, key=lambda x: x.lower().replace("_", " "))
print(nii_files)
print()

# get all csv files (labels)
csv_files = filter(lambda x: x[-4:] == '.csv', all_files)
csv_files = list(csv_files)
csv_files = sorted(csv_files, key=lambda x: x.lower().replace("_", " "))
print(csv_files)

test_images = list()
test_labels = list()


# read-in each image and its data
for item in nii_files:
    file_name = item
    image = nib.load(file_name)
    image_data = image.get_fdata()
    test_images.append(image_data)

# read-in each data frame of labels
for item in csv_files:
    file_name = item
    df = pd.read_csv(file_name)
    test_labels.append(df)

['S130.nii', 'S131.nii', 'S132.nii', 'S133.nii', 'S134.nii', 'S135.nii', 'S136.nii', 'S137.nii', 'S138.nii', 'S139.nii', 'S140.nii', 'S141.nii', 'S142.nii']

['S130_labels.csv', 'S131_labels.csv', 'S132_labels.csv', 'S133_labels.csv', 'S134_labels.csv', 'S135_labels.csv', 'S136_labels.csv', 'S137_labels.csv', 'S138_labels.csv', 'S139_labels.csv', 'S140_labels.csv', 'S141_labels.csv', 'S142_labels.csv']


Convert data to appropriate type/format for input into CNN by stacking input tensors

In [ ]:
test_images = tf.stack(test_images)
test_labels = np.array(test_labels)
test_labels = tf.stack(test_labels)

print("Test images shape:")
print(test_images.get_shape())
print("Test labels shape:")
print(test_labels.get_shape())

Test images shape:
(13, 128, 128, 64)
Test labels shape:
(13, 1, 9)


In [ ]:
os.chdir("/content/gdrive/My Drive/FYP Data")

LOAD MODEL

In [ ]:
# for "Copy of Copy of 3D CNN v14_moreRAM_moreConv_batch3 (best)_ALL_DATA.ipynb"
#loaded_model = keras.models.load_model("/content/gdrive/My Drive/FYP Data/CNN MODEL USING ALL DATA")

# for "Copy of 3D CNN v8d_re-run_with_updates.ipynb"
#loaded_model = keras.models.load_model("/content/gdrive/My Drive/FYP Data/CNN MODEL 8 updated")

loaded_model = keras.models.load_model("/content/gdrive/My Drive/FYP Data/CNN MODEL v1 ALL DATA")

EVALUATE MODEL

In [ ]:
# Returns the loss value & metrics values for the model in test mode.
test_loss, test_mae, test_mse, test_rmse = loaded_model.evaluate(test_images,  test_labels, verbose=2)

1/1 - 10s - loss: 1118.2905 - mean_absolute_error: 31.7965 - mean_squared_error: 1118.2905 - root_mean_squared_error: 33.4408 - 10s/epoch - 10s/step


TEST MODEL

In [ ]:
# Test data
unseen_images = test_images

# Use model to make predictions
predicted_endpoints = loaded_model.predict(unseen_images)

# show the inputs and predicted outputs
for i in range(len(unseen_images)):
 print("endpoint predicted coordinates=%s, endpoint ground truth coordinates=%s" % (predicted_endpoints[i], test_labels[i]))
 print(" ")

1/1 [==============================] - 8s 8s/step
endpoint predicted coordinates=[19.063774 30.749224 12.485241 29.922047 21.744967 15.897197 32.50592
 36.302456 15.449302], endpoint ground truth coordinates=tf.Tensor([[52 77 26 70 50 34 74 78 39]], shape=(1, 9), dtype=int64)
 
endpoint predicted coordinates=[18.819403 29.99226  12.351194 29.391651 21.489187 15.661915 31.851114
 35.70296  15.069947], endpoint ground truth coordinates=tf.Tensor([[55 81 28 76 64 34 86 82 38]], shape=(1, 9), dtype=int64)
 
endpoint predicted coordinates=[18.447815 30.342249 12.150175 29.54179  21.319092 15.754082 32.15341
 35.522118 15.186769], endpoint ground truth coordinates=tf.Tensor([[47 75 33 64 61 39 73 81 41]], shape=(1, 9), dtype=int64)
 
endpoint predicted coordinates=[19.200891 30.615528 12.507575 29.724869 21.779373 15.675943 32.608963
 36.12224  15.195457], endpoint ground truth coordinates=tf.Tensor([[51 71 36 70 45 36 84 64 41]], shape=(1, 9), dtype=int64)
 
endpoint predicted coordinates=[

REVERSE TRANSFORM

In [ ]:
# get predicted outputs into data frame for post-processing
predicted_endpoints # convert this to pandas data frame along with test labels and do analysis on that.
df_pred = pd.DataFrame(predicted_endpoints, columns=['x_1', 'y_1', 'z_1', 'x_2', 'y_2', 'z_2', 'x_3', 'y_3', 'z_3'])
df_pred

,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3
0,19.063774,30.749224,12.485241,29.922047,21.744967,15.897197,32.505920,36.302456,15.449302
1,18.819403,29.992260,12.351194,29.391651,21.489187,15.661915,31.851114,35.702961,15.069947
2,18.447815,30.342249,12.150175,29.541790,21.319092,15.754082,32.153412,35.522118,15.186769
3,19.200891,30.615528,12.507575,29.724869,21.779373,15.675943,32.608963,36.122238,15.195457
4,19.134291,30.450962,12.490331,29.647980,21.650270,15.737363,32.292469,36.088490,15.243904
5,19.128902,30.898611,12.589166,29.947826,21.936728,15.872730,32.561104,36.289547,15.427897
6,18.383015,30.884211,12.194110,29.829134,21.361256,15.857288,32.458946,35.703770,15.393233
7,18.838352,30.935064,12.495258,29.979067,21.604368,15.913667,32.528435,36.084091,15.525405
8,18.551298,30.541607,12.122112,29.594074,21.376562,15.716597,32.300953,35.666973,15.256326
9,18.832945,30.980299,12.320162,29.944483,21.726425,15.882194,32.741547,36.139668,15.464694


In [ ]:
# reverse the transformation
df_pred["x_1"] = df_pred["x_1"]*(512/128)
df_pred["y_1"] = df_pred["y_1"]*(512/128)

df_pred["x_2"] = df_pred["x_2"]*(512/128)
df_pred["y_2"] = df_pred["y_2"]*(512/128)

df_pred["x_3"] = df_pred["x_3"]*(512/128)
df_pred["y_3"] = df_pred["y_3"]*(512/128)

# z normalisation depends on number of slices in CT stack which varies between patients. Thus need number of scans in each case:
depths = [411, 121, 327, 481, 378, 483, 478, 229, 455, 460, 80, 260, 114]; # acquired using script to get depth of each CT scan in test data set

for i in np.arange(len(depths)):
  df_pred["z_1"][i] = df_pred["z_1"][i]*(depths[i]/64)
  df_pred["z_2"][i] = df_pred["z_2"][i]*(depths[i]/64)
  df_pred["z_3"][i] = df_pred["z_3"][i]*(depths[i]/64)

df_pred

<ipython-input-10-97cab39a2fee>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred["z_2"][i] = df_pred["z_2"][i]*(depths[i]/64)
<ipython-input-10-97cab39a2fee>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred["z_3"][i] = df_pred["z_3"][i]*(depths[i]/64)


,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3
0,76.255096,122.996895,80.178657,119.688187,86.979866,102.089811,130.023682,145.209824,99.213484
1,75.277611,119.969040,23.351477,117.566605,85.956749,29.610808,127.404457,142.811844,28.491619
2,73.791260,121.368996,62.079801,118.167160,85.276367,80.493511,128.613647,142.088470,77.594895
3,76.803566,122.462112,94.002244,118.899475,87.117493,117.814512,130.435852,144.488953,114.203360
4,76.537163,121.803848,73.771016,118.591919,86.601082,92.948799,129.169876,144.353958,90.034309
5,76.515610,123.594444,95.008860,119.791306,87.746910,119.789511,130.244415,145.158188,116.432414
6,73.532059,123.536842,91.074758,119.316536,85.445023,118.434122,129.835785,142.815079,114.968211
7,75.353409,123.740257,44.709596,119.916267,86.417473,56.941089,130.113739,144.336365,55.551840
8,74.205193,122.166428,86.180642,118.376297,85.506248,111.735179,129.203812,142.667892,108.462941
9,75.331779,123.921196,88.551163,119.777931,86.905701,114.153266,130.966187,144.558670,111.152488


ORIGINAL LABEL DATA:

In [ ]:
df_true = pd.read_csv("/content/gdrive/My Drive/FYP Data/test_labels_df_un_norm_all_data.csv")
df_true

,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3
0,307,206,170,200,279,221,311,295,252
1,324,221,52,254,304,65,326,346,72
2,300,187,171,245,256,198,324,291,209
3,284,205,267,181,282,274,257,335,306
4,290,207,153,197,301,198,316,326,218
5,270,195,265,184,277,260,333,306,283
6,294,149,250,201,233,274,322,255,299
7,298,177,106,179,283,135,327,312,142
8,269,169,214,223,253,262,334,263,260
9,274,175,222,207,254,270,301,270,291


ERROR ANALYSIS

In [ ]:
# ERROR IN X, Y, Z
import plotly.graph_objs as go

# Endpoint 1 error at each label (x,y,z and overall)
x1_error = abs(np.asarray(df_true["x_1"]) - np.asarray(df_pred["x_1"]))
y1_error = abs(np.asarray(df_true["y_1"]) - np.asarray(df_pred["y_1"]))
z1_error = abs(np.asarray(df_true["z_1"]) - np.asarray(df_pred["z_1"]))

x2_error = abs(np.asarray(df_true["x_2"]) - np.asarray(df_pred["x_2"]))
y2_error = abs(np.asarray(df_true["y_2"]) - np.asarray(df_pred["y_2"]))
z2_error = abs(np.asarray(df_true["z_2"]) - np.asarray(df_pred["z_2"]))

x3_error = abs(np.asarray(df_true["x_3"]) - np.asarray(df_pred["x_3"]))
y3_error = abs(np.asarray(df_true["y_3"]) - np.asarray(df_pred["y_3"]))
z3_error = abs(np.asarray(df_true["z_3"]) - np.asarray(df_pred["z_3"]))

x_error = (1/3)*(x1_error + x2_error + x3_error)
y_error = (1/3)*(y1_error + y2_error + y3_error)
z_error = (1/3)*(z1_error + z2_error + z3_error)

# x
X = np.arange(test_labels.get_shape()[0]) + 1

# create plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=X, y=x_error, name="Overall MAE in x"))
fig.add_trace(go.Scatter(x=X, y=y_error, name="Overall MAE in y"))
fig.add_trace(go.Scatter(x=X, y=z_error, name="Overall MAE in z"))

fig.update_layout(
   xaxis_title="Test data sample",
   yaxis_title="Error (pixels)",
   font=dict(
      size=18))

# Display the plot
fig.show()

In [ ]:
# ERROR @ EP 1, 2, 3

# absolute error at endpoint 1
x1_error = abs(np.asarray(df_true["x_1"]) - np.asarray(df_pred["x_1"]))
y1_error = abs(np.asarray(df_true["y_1"]) - np.asarray(df_pred["y_1"]))
z1_error = abs(np.asarray(df_true["z_1"]) - np.asarray(df_pred["z_1"]))
#EP1_error = (1/3)*(x1_error + y1_error + z1_error)
EP1_error = np.sqrt(x1_error**2 + y1_error**2 + z1_error**2)

# absolute error at endpoint 2
x2_error = abs(np.asarray(df_true["x_2"]) - np.asarray(df_pred["x_2"]))
y2_error = abs(np.asarray(df_true["y_2"]) - np.asarray(df_pred["y_2"]))
z2_error = abs(np.asarray(df_true["z_2"]) - np.asarray(df_pred["z_2"]))
#EP2_error = (1/3)*(x2_error + y2_error + z2_error)
EP2_error = np.sqrt(x2_error**2 + y2_error**2 + z2_error**2)

# absolute error at endpoint 3
x3_error = abs(np.asarray(df_true["x_3"]) - np.asarray(df_pred["x_3"]))
y3_error = abs(np.asarray(df_true["y_3"]) - np.asarray(df_pred["y_3"]))
z3_error = abs(np.asarray(df_true["z_3"]) - np.asarray(df_pred["z_3"]))
#EP3_error = (1/3)*(x3_error + y3_error + z3_error)
EP3_error = np.sqrt(x3_error**2 + y3_error**2 + z3_error**2)

# Create Line plot
X = np.arange(test_labels.get_shape()[0]) + 1

# create plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=X, y=EP1_error, name="Overall MAE at endpoint 1"))
fig.add_trace(go.Scatter(x=X, y=EP2_error, name="Overall MAE at endpoint 2"))
fig.add_trace(go.Scatter(x=X, y=EP3_error, name="Overall MAE at endpoint 3"))

fig.update_layout(
   xaxis_title="Test data sample",
   yaxis_title="Error (pixels)",
   font=dict(
      size=18))

# Display the plot
fig.show()

In [ ]:
# Overall error at each label
# Create Line plot
X = np.arange(test_labels.get_shape()[0]) + 1
overall_error = (1/3)*(abs(EP1_error) + abs(EP2_error) + abs(EP3_error))

# create plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=X, y=overall_error, name="Overall absolute error"))

fig.update_layout(
   xaxis_title="Test data sample",
   yaxis_title="Error (pixels)",
   font=dict(size=18),
   showlegend=True)


# Display the plot
fig.show()

In [ ]:
n = 13 # number of samples
test_errors = [260.3671, 272.3502, 260.3618, 279.3687, 261.57, 273.0564, 263.7235, 243.0902, 255.851, 255.5887, 224.5457, 223.2282, 242.1677]
mean = (1/n)*sum(test_errors)
print("mean =", mean)
st_dev = np.std(test_errors)
print("standard deviation =", st_dev)

mean = 255.0207076923077
standard deviation = 16.738264986434363
